<a href="https://colab.research.google.com/github/custom-hyper/CoinGeckoAPI/blob/main/CoinGecko_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#@title Install Libraries { display-mode: "code" }
!pip install plotly
!pip install pytrends
!pip install pycoingecko
!pip install pandas
!pip install tweepy
!pip install requests
!pip install pycoingecko


  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=7b7c82ccdda496d3029147ec4276dcaae11bc52eb249b1f14a7da5ec09d52a11
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends


# CoinGecko API Statistics Report

In [ ]:
#@title Available Currencies { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from pycoingecko import CoinGeckoAPI


In [59]:
#General Information table

def currency_stats(url):
    """
    Generates a list of the currencies on CoinGecko
    Args: url of the CoinGecko website

    """
    frames = []
    response = requests.get(url)
    data = response.json()

    df_currency = pd.DataFrame(data, columns = ['id'])
    """
    print('Stats: ')
    print(df_currency.info(verbose=True))
    print(df_currency.isnull().sum())
    print(data)

    print(df_currency.describe())
    """
    return df_currency

def main():

    url = "https://api.coingecko.com/api/v3/simple/supported_vs_currencies"
    currency_stats(url)
    
if __name__ == "__main__":
    main()



In [61]:
df_currency

NameError: ignored

In [71]:
#@title Available Coins { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

#General Information table

def coins_stats(url):
    """
    Generates a list of the coins on CoinGecko
    Args: url of the CoinGecko website

    """
    frames = []
    response = requests.get(url)
    data = response.json()

    
    token_list = pd.DataFrame(data)

    print('Stats: ')
    print(token_list.info(verbose=True))
    print(token_list.isnull().sum())
    print(token_list.describe())
   
    return token_list 

def prop_id():
    """
    Generates a list of the coins on CoinGecko
    Args: url of the CoinGecko website

    """
    token_list_clean = pd.DataFrame() 
    token_list_clean['CoinGecko_id'] = token_list['id']
    symbol = token_list['symbol'].str.upper()
    name = token_list['name'] 

    token_list_clean['prop_id'] = symbol + '.' + name.str.replace(" ", "")
    token_list_clean['symbol'] = symbol
    token_list_clean['name'] = name

    token_list_clean = token_list_clean.dropna()
    token_list_clean.reset_index(drop=True, inplace=True)

    return token_list_clean

def main():
  url = "https://api.coingecko.com/api/v3/coins/list"

  coins_stats(url)
  prop_id()  

  token_list_clean.head()

if __name__ == "__main__":
    main()

,CoinGecko_id,prop_id,symbol,name
0,01coin,ZOC.01coin,ZOC,01coin
1,0-5x-long-algorand-token,ALGOHALF.0.5XLongAlgorandToken,ALGOHALF,0.5X Long Algorand Token
2,0-5x-long-altcoin-index-token,ALTHALF.0.5XLongAltcoinIndexToken,ALTHALF,0.5X Long Altcoin Index Token
3,0-5x-long-ascendex-token-token,ASDHALF.0.5XLongAscendExTokenToken,ASDHALF,0.5X Long AscendEx Token Token
4,0-5x-long-bitcoin-cash-token,BCHHALF.0.5XLongBitcoinCashToken,BCHHALF,0.5X Long Bitcoin Cash Token


# CoinGecko API Extraction

In [37]:
#@title  Download Coins Data Reports { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import logging
import sys
from tqdm.notebook import tqdm

url = "https://api.coingecko.com/api/v3/coins/list"
frames = []
response = requests.get(url)
data = response.json()
token_list = pd.DataFrame(data)
id = token_list.id
id


#loop data extraction to token list
try:

  start = 2001
  ends =  3000

  for i in id[start : ends]:

    response = requests.get("https://api.coingecko.com/api/v3/coins/{}".format(i))
    output = response.text
    json_parsed = json.loads(output)
    print(json_parsed)

    #Convert data to list of dataframes
    full_table = pd.json_normalize(json_parsed)       
    #results = [full_table]
    frames.append(full_table)

    #Slow down requests

    time.sleep(60/50)
    #Status report
    logging.info('Page '+ str(i) + ' – Rows ' + ' – source: '+ str(url))
except ConnectionError:
    time.sleep(60)
 
single = pd.concat(frames) #Careful, this line must be outside of the for loop

print(single)
path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output/"
single.to_csv(path + 'CoinGecko_coins_{}-{}.csv'.format(start,ends))



{'id': 'bomberman', 'symbol': '$bomb', 'name': 'Bomberman', 'asset_platform_id': 'binance-smart-chain', 'platforms': {'binance-smart-chain': '0xf8d5153839d8ace20ee6d262c24c8c168e057b88'}, 'block_time_in_minutes': 0, 'hashing_algorithm': None, 'categories': ['Binance Smart Chain Ecosystem'], 'public_notice': None, 'additional_notices': [], 'localization': {'en': 'Bomberman', 'de': 'Bomberman', 'es': 'Bomberman', 'fr': 'Bomberman', 'it': 'Bomberman', 'pl': 'Bomberman', 'ro': 'Bomberman', 'hu': 'Bomberman', 'nl': 'Bomberman', 'pt': 'Bomberman', 'sv': 'Bomberman', 'vi': 'Bomberman', 'tr': 'Bomberman', 'ru': 'Bomberman', 'ja': 'Bomberman', 'zh': 'Bomberman', 'zh-tw': 'Bomberman', 'ko': 'Bomberman', 'ar': 'Bomberman', 'th': 'Bomberman', 'id': 'Bomberman', 'cs': 'Bomberman', 'da': 'Bomberman', 'el': 'Bomberman', 'hi': 'Bomberman', 'no': 'Bomberman', 'sk': 'Bomberman', 'uk': 'Bomberman', 'he': 'Bomberman', 'fi': 'Bomberman', 'bg': 'Bomberman', 'hr': 'Bomberman', 'lt': 'Bomberman', 'sl': 'Bombe

KeyboardInterrupt: ignored

# Aggregating Output

In [ ]:
import os
import glob
import pandas as pd
import sys
from pathlib import Path




path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output"


Cleaned_data_file = 'Bulk_CoinGecko_coins.csv'

df = pd.DataFrame()

extension = 'xlsx'
#dataset_location = os.chdir(r'D:\investing_programs\datasets\CoinMetrics')
all_filenames = []
all_filenames = [i for i in glob.glob('/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output/*.{}'.format(extension))]

#combine all files in the list
for i in all_filenames:
    df = pd.read_excel(i.read(),engine='openpyxl')   
    print(df)

combined_csv = df.append([pd.read_excel(f) for f in all_filenames])


combined_csv.to_csv( path +'\\'+ Cleaned_data_file, index=False, encoding='utf-8-sig')


print(combined_csv.head())

sys.exit()
